# MNIST

In [1]:
import cudf

In [2]:
import cuml
from cuml.svm import SVC

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [4]:
from sklearn.datasets import fetch_openml

In [5]:
mnist = fetch_openml('mnist_784')

In [6]:
mnist['data']

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
mnist['target']

0        5
1        0
2        4
3        1
4        9
        ..
69995    2
69996    3
69997    4
69998    5
69999    6
Name: class, Length: 70000, dtype: category
Categories (10, object): ['0', '1', '2', '3', ..., '6', '7', '8', '9']

In [8]:
train = cudf.DataFrame(mnist['data'])
import cupy as cp
X = train
y = cudf.DataFrame(mnist['target'])
y = y.astype('int').to_cupy()
y = y.reshape(70000, )

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 612)

In [10]:
SVModel_GPU = SVC()

%timeit SVModel_GPU.fit(X_train, y_train)

y_pred_gpu = SVModel_GPU.predict(X = X_test)

6.7 s ± 2.31 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
from sklearn.metrics import accuracy_score

print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test.get(), y_pred_gpu.to_numpy())))

Model accuracy score: 0.9791


In [12]:
# Print the Confusion Matrix

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test.get(), y_pred_gpu.to_numpy())

print('Confusion matrix\n\n', cm)

Confusion matrix

 [[1374    0    3    0    0    2    3    0    4    1]
 [   0 1553    6    5    1    0    1    2    2    1]
 [   0    3 1395    4    4    1    0    4    4    1]
 [   0    0    6 1400    1   10    0    8   13    6]
 [   0    2    4    0 1349    1    5    2    1   10]
 [   4    0    1   13    4 1256    6    0    3    3]
 [   5    2    2    0    4    4 1316    0    5    0]
 [   0    5    7    0    6    1    0 1372    3   13]
 [   1    6    4    4    1    6    3    1 1327    3]
 [   2    1    1   11   17    4    0    9    6 1366]]


In [13]:
from sklearn.metrics import classification_report

print(classification_report(y_test.get(), y_pred_gpu.to_numpy()))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1387
           1       0.99      0.99      0.99      1571
           2       0.98      0.99      0.98      1416
           3       0.97      0.97      0.97      1444
           4       0.97      0.98      0.98      1374
           5       0.98      0.97      0.98      1290
           6       0.99      0.98      0.99      1338
           7       0.98      0.98      0.98      1407
           8       0.97      0.98      0.97      1356
           9       0.97      0.96      0.97      1417

    accuracy                           0.98     14000
   macro avg       0.98      0.98      0.98     14000
weighted avg       0.98      0.98      0.98     14000



# EMINST

In [14]:
emnist = fetch_openml('EMNIST_Balanced')

In [15]:
emnist['data']

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131595,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
131596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
131597,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
131598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
emnist['target']

0         45
1         36
2         43
3         15
4          4
          ..
131595    45
131596    15
131597     1
131598    26
131599    33
Name: class, Length: 131600, dtype: category
Categories (47, object): ['0', '1', '2', '3', ..., '43', '44', '45', '46']

In [17]:
etrain = cudf.DataFrame(emnist['data'])
etrain

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131595,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
131596,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
131597,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
131598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
import cupy as cp

In [19]:
eX = etrain
ey = cudf.DataFrame(emnist['target'])
ey = ey.astype('int').to_cupy()
ey

array([[45],
       [36],
       [43],
       ...,
       [ 1],
       [26],
       [33]])

In [20]:
ey = ey.reshape(131600, )

In [21]:
X_train, X_test, y_train, y_test = train_test_split(eX, ey, test_size = 0.2, random_state = 612)

In [22]:
SVModel_GPU = SVC()

%timeit SVModel_GPU.fit(X_train, y_train)

y_pred_gpu = SVModel_GPU.predict(X = X_test)

1min 31s ± 668 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
from sklearn.metrics import accuracy_score

print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test.get(), y_pred_gpu.to_numpy())))

Model accuracy score: 0.8406


In [24]:
# Print the Confusion Matrix

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test.get(), y_pred_gpu.to_numpy())

print('Confusion matrix\n\n', cm)

Confusion matrix

 [[365   0   0 ...   0   0   0]
 [  0 437   1 ...   0   2   1]
 [  0   0 448 ...   2   1   0]
 ...
 [  0   1   1 ... 287   2   5]
 [  0   0   0 ...   1 510   6]
 [  0   1   0 ...   0   4 447]]


In [25]:
from sklearn.metrics import classification_report

print(classification_report(y_test.get(), y_pred_gpu.to_numpy()))

              precision    recall  f1-score   support

           0       0.60      0.67      0.63       545
           1       0.53      0.75      0.62       581
           2       0.89      0.86      0.87       521
           3       0.95      0.92      0.94       557
           4       0.89      0.89      0.89       575
           5       0.89      0.81      0.85       541
           6       0.90      0.88      0.89       567
           7       0.96      0.97      0.96       562
           8       0.88      0.87      0.87       567
           9       0.65      0.88      0.75       560
          10       0.91      0.91      0.91       567
          11       0.86      0.89      0.88       555
          12       0.91      0.93      0.92       575
          13       0.85      0.90      0.87       549
          14       0.95      0.90      0.92       554
          15       0.68      0.52      0.59       584
          16       0.90      0.91      0.91       584
          17       0.90    